In [1]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver
from bs4 import BeautifulSoup 
from tqdm.notebook import tqdm
import time
import re

# 카테고리 담기

In [2]:
browser = webdriver.Chrome('C:\chrome\chromedriver.exe')
url = "https://www.idus.com/c/category/3" # 공예
browser.get(url)
time.sleep(3)

html = browser.page_source
soup=BeautifulSoup(html, 'html.parser')

In [3]:
category_lists = soup.select('div.inner-w.mobile-full ul.side_nav__sublist li.side_nav__submenu a')
print(len(category_lists))
# print(category_lists[0].text)
# print(category_lists[34].text)

for i in range(0,12):
    print(category_lists[i].text)

35
캔들, 디퓨저, 방향제
실크스크린, 염색
발향, 룸스프레이
자수, 재봉, 펠트
뜨개, 마크라메
가죽
금속, 악세사리
네온사인, 와이어
유리
목공
도자기
기타


In [6]:
cat_data = [] # 빈집 만들기 #공예 해당하는 것
for i in range(0,12):
    cat_list = category_lists[i].text
    cat_num = i+15
    cat_data.append([cat_list, cat_num])  
    
columns = ['카테고리','category_num'] 
# "https://www.idus.com/c/category/3" / 카테고리 넘버 입력시 그 페이지로 이동함
cat_df = pd.DataFrame(cat_data, columns=columns)
cat_df

,카테고리,category_num
0,"캔들, 디퓨저, 방향제",15
1,"실크스크린, 염색",16
2,"발향, 룸스프레이",17
3,"자수, 재봉, 펠트",18
4,"뜨개, 마크라메",19
5,가죽,20
6,"금속, 악세사리",21
7,"네온사인, 와이어",22
8,유리,23
9,목공,24


# url 사용자 함수 만들기

In [95]:
import time
import pandas as pd
import re

def collect_url(page):
    start = time.time()
    url = f'https://www.idus.com/c/category/3/{page}'
    #  url 접속
    browser = webdriver.Chrome('C:\chrome\chromedriver.exe')
    browser.get(url)
    time.sleep(2)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
       
    # 현재 문서 높이를 가져와서 저장
    prev_height = browser.execute_script("return document.body.scrollHeight")

    while True:
        # 스크롤을 가장 아래로 내림
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = browser.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

        
    #url들 수집
    url_data =[]
    page = f'{page}'

    # URL_raw 크롤링 시작
    articles = ".ui_card__title" # 중간에 띄어쓰기는 .으로 대체
    article_raw = browser.find_elements_by_css_selector(articles)
    article_raw[0]\

    # 1초 쉬어 주고,
    time.sleep(1)

    # 크롤링한 url 정제 시작
    for article in article_raw:
        url = article.get_attribute('href')  
        title = article.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", title)
        url_data.append([page, url, title])

    # 데이터 프레임 만들기
    columns = ['category_num', 'url', 'title']
    df = pd.DataFrame(url_data, columns = columns)

    # 데이터 프레임 저장하기   
    df_name = './re_url/idus_{}.xlsx'.format(page) # url 폴더 생성하거나 지워서 run
    df.to_excel(df_name, index=False) #,engine='openpyxl'
    
    end = time.time() - start
    print("used time: %dm%ds" % divmod(end,60))
    return(df)

    

In [ ]:
for i in range(15,27):
    collect_url(i)    

In [ ]:
# url 합치기
url_df = pd.DataFrame([])


for i in range(15,27):
    folder = './idus/'
    f_list = 'idus_{}.xlsx'.format(i)
    f_path = folder+f_list
    temp = pd.read_excel(f_path)
    url_df = url_df.append(temp)  
url_df.info()

In [ ]:
url_df.to_excel('./idus/url_all.xlsx', index=False)

In [109]:
url_df=pd.read_excel('./idus/url_all.xlsx', index=False)
url_df

,category_num,url,title
0,15,https://www.idus.com/c/class/4068,향기로운 플라워타블렛과 섬유향수만들기
1,15,https://www.idus.com/c/class/3350,"반짝반짝, 크리스탈캔들 패브릭미스트 클래스"
2,15,https://www.idus.com/c/class/964,경복궁역 향기 가득 플라워 왁스 타블렛 만들기
3,15,https://www.idus.com/c/class/4552,달콤 디저트 까눌레 캔들 톰과제리 치즈캔들 만들기
4,15,https://www.idus.com/c/class/3894,건대입구 젤캔들 홀더와 티라이트4개 원데이클래스
...,...,...,...
449,26,https://www.idus.com/c/class/1998,부산 라탄원목트레이
450,26,https://www.idus.com/c/class/1896,힐링라탄 감성소품 미니스탠드 원데이클래스
451,26,https://www.idus.com/c/class/1688,힐링라탄 타원트레이
452,26,https://www.idus.com/c/class/729,레진머리핀 3개 내 맘에 쏘 옥 드는 레진 머리핀을 만들자


# 댓글 내용 크롤링(연습)

In [103]:
url = 'https://www.idus.com/c/class/3810'# 댓글이 60개가량 있는 글 하나를 예시로 가져옴
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # 윈도우는 "chromedriver.exe"
driver.get(url)

In [104]:
driver.find_element_by_link_text("참여후기").click( )
time.sleep(1)
# n: 반복할 횟수 설정
n = 4
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    try:
        time.sleep(2)
        driver.find_element_by_css_selector("#edu-review > div.pagination > div > button > span").click( )
        i = i+1
       
    except:
        break

In [35]:
# 제목 크롤링 시작
overlays = "h2.sticky_aside__title"                                 
tit = driver.find_element_by_css_selector(overlays)          # title
title = tit.text
title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", "", title)

In [36]:
title

'전문 네온공예교육 스무디 20diy 강남점'

In [44]:
comments = driver.find_elements_by_css_selector('ul.list-style-review>li')
len(comments)

23

In [46]:
comments

[<selenium.webdriver.remote.webelement.WebElement (session="12cdd7b2086c18a148c1f23714868cc0", element="2b6cb7d4-cd98-4d98-bd3a-d945e3444936")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12cdd7b2086c18a148c1f23714868cc0", element="85263397-431f-42fe-8e27-f2ece1d02871")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12cdd7b2086c18a148c1f23714868cc0", element="f634b2dc-713d-4575-abe7-944efb8a5c40")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12cdd7b2086c18a148c1f23714868cc0", element="8ce20311-2464-44f8-8853-49b4ad0f0cc4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12cdd7b2086c18a148c1f23714868cc0", element="53a5bd2a-40bb-4f03-a19f-e1a373faedf4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12cdd7b2086c18a148c1f23714868cc0", element="d3318a17-94ca-4954-a2ec-69f6ee3875bf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="12cdd7b2086c18a148c1f23714868cc0", element="e4d82e1c-9650-454d-88f3-91

In [63]:
comment=comments[0]

In [93]:
nickname =comment.find_elements_by_css_selector('a > div.clf > div > div.txt-block > em')[0].text
nickname

'서지혜'

In [91]:
content = comment.find_elements_by_css_selector('a > div.area-txt > div > span')[0].text
content

'위치가 좋인 찾아가기도 쉽고~아주 친절하게 알려주세요^-^1대1로 배우니 빠르게 진행되어 어려운 도안이었지만 시간 오버되지 않았네요! 감사합니다~ 기회되면 다른체험도 하러 재방문 할께요!너무나 만족스런 작품이 나와 기분 좋은 주말 저녁이었답니다^-^'

In [94]:
like = comment.find_elements_by_css_selector('a > div.clf > span')[0].get_attribute('data-value')
like

'5'

In [99]:
date = comment.find_elements_by_css_selector('a > div.clf > div > div.txt-block > time')[0].text
date

'2020년 6월 12일'

In [105]:
comments = driver.find_elements_by_css_selector('ul.list-style-review>li')
for comment in comments:
    # 제목 크롤링 시작
    overlays = "h2.sticky_aside__title"                                 
    tit = driver.find_element_by_css_selector(overlays)          # title
    title = tit.text
    title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", "", title)
    
    nick = comment.find_elements_by_css_selector('a > div.clf > div > div.txt-block > em')[0].text
    nick = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", "", nick)
    
    date = comment.find_elements_by_css_selector('a > div.clf > div > div.txt-block > time')[0].text
    
    like = comment.find_elements_by_css_selector('a > div.clf > span')[0].get_attribute('data-value')
    
    content = comment.find_elements_by_css_selector('a > div.area-txt > div > span')[0].text
    content = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", "", content)
    
    print(title,nick, date, like, content,sep='|')

전문 네온공예교육 스무디 20diy 강남점|서지혜|2021년 4월 12일|5|위치가 좋인 찾아가기도 쉽고아주 친절하게 알려주세요1대1로 배우니 빠르게 진행되어 어려운 도안이었지만 시간 오버되지 않았네요 감사합니다 기회되면 다른체험도 하러 재방문 할께요너무나 만족스런 작품이 나와 기분 좋은 주말 저녁이었답니다
전문 네온공예교육 스무디 20diy 강남점|유혜린|2021년 4월 11일|5|어떻게 만들건지 같이 도안짜주시고 친절하게 잘알려주셨어여 만든 작품도 맘에 들고 담에 또 방문해서 만들고싶네요 재밌고 즐거운 경험이었습니다
전문 네온공예교육 스무디 20diy 강남점|한유진|2021년 3월 15일|4|이제 곧 이사가는데 이사갈 집에 부엌이랑 방에 걸어두면 좋을 것 같아서 중학생 막내랑 같이가서 만들어보았어용생각보다 너무 어려웠긴 했지만.. 결과물보니 넘 예쁘드라구요글자색이랑 캐릭터색에 입힐 네온선이 하나로는 부족할 것 같아서 네온선색상 각각 2개씩 추가해서 10000원씩 2만원 추가비용내고 만들었답니다아이디어스를 통해 만들기체험은 처음이라 리뷰도 처음 달아보네요 번창하세요
전문 네온공예교육 스무디 20diy 강남점|율s|2021년 2월 14일|4|클래스 자체는 정말 새롭고 재미있어서 또 가고 싶어졌습니다 도안도 가서 직접 고르거나 미리 골라간 뒤 인쇄할 수 있어 매우 편합니다.다만, 만들기에 너무 열중한 나머지 현장에서는 미처 인식하지 못한 점이 있었는데, 환기가 전혀 안 된다는 것입니다. 현관문을 닫아놓는 것은 물론 창문도 없는 좁은 방에 체험생 6명과 선생님 한 분까지 총 7명이 있으니 공기가 답답해질 수 밖에 없었습니다. 이를 미리 인지했다면 환기를 건의 드렸을텐데 답답함도 잊고 만들기에 집중할만큼 너무 재미있었습니다
전문 네온공예교육 스무디 20diy 강남점|홍지윤|2021년 2월 7일|5|작가님이 친절하게 알려주세요친구랑 가서 좋은 추억 만들고 왔습니당ㅎㅎㅎ
전문 네온공예교육 스무디 20diy 강남점|서유현|2021년 1월 28일|5|여태껏 가본 원데이 클

# 합치기

In [2]:
# url 불러오기
url_df=pd.read_excel('./idus/url_all.xlsx',engine='openpyxl')
url_df

,category_num,url,title
0,15,https://www.idus.com/c/class/4068,향기로운 플라워타블렛과 섬유향수만들기
1,15,https://www.idus.com/c/class/3350,"반짝반짝, 크리스탈캔들 패브릭미스트 클래스"
2,15,https://www.idus.com/c/class/964,경복궁역 향기 가득 플라워 왁스 타블렛 만들기
3,15,https://www.idus.com/c/class/4552,달콤 디저트 까눌레 캔들 톰과제리 치즈캔들 만들기
4,15,https://www.idus.com/c/class/3894,건대입구 젤캔들 홀더와 티라이트4개 원데이클래스
...,...,...,...
449,26,https://www.idus.com/c/class/1998,부산 라탄원목트레이
450,26,https://www.idus.com/c/class/1896,힐링라탄 감성소품 미니스탠드 원데이클래스
451,26,https://www.idus.com/c/class/1688,힐링라탄 타원트레이
452,26,https://www.idus.com/c/class/729,레진머리핀 3개 내 맘에 쏘 옥 드는 레진 머리핀을 만들자


In [ ]:
import time
import re
import pandas as pd
from tqdm.notebook import tqdm 

# url 불러오기
url_load = pd.read_excel("./idus/url_all.xlsx",engine='openpyxl')
num_list = len(url_load)
print(num_list) # 총 url 갯수
url_load

start = time.time() 

# number = len(url_load)  # 수집할 글 갯수
# number = num_list

title_list = []
nickname_list = []
datetime_list = []
like_list = []
content_list = []

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, 454)): # 끊긴 경우 여기 0대신해 시작 숫자만 변경하면 됨
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
    driver.get(url_load.url[i])
    time.sleep(2)
    
    try:
        
        target_info = {} 
        
        try:
             # '참여후기' 클릭
            driver.find_element_by_link_text("참여후기").click( )
            time.sleep(2)
        
            # n: 반복할 횟수 설정, '더보기' 클릭
            n = 4
            i = 0
            while i < n: # 이 조건이 만족되는 동안 반복 실행
                try:
                    time.sleep(2)
                    driver.find_element_by_css_selector("#edu-review > div.pagination > div > button > span").click( ) #더보기
                    i = i+1
                except:
                    break
                    
        except:
            pass

        comments = driver.find_elements_by_css_selector('ul.list-style-review>li')
        for comment in comments:                     
            title = driver.find_element_by_css_selector("h2.sticky_aside__title").text        
            title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", "", title)
            title_list.append(title)

            nickname = comment.find_elements_by_css_selector('a > div.clf > div > div.txt-block > em')[0].text
            nickname = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", "", nickname)
            nickname_list.append(nickname)

            datetime = comment.find_elements_by_css_selector('a > div.clf > div > div.txt-block > time')[0].text
            datetime_list.append(datetime)

            like = comment.find_elements_by_css_selector('a > div.clf > span')[0].get_attribute('data-value')
            like_list.append(like)

            content = comment.find_elements_by_css_selector('a > div.area-txt > div > span')[0].text
            content = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", "", content)
            content_list.append(content)      
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except Exception as e:
        driver.close()
        print('실패', e, type(e))
        continue
        
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 200 or 300 or 400 :
        columns = ['title', 'nickname', 'datetime', 'like', 'content']
        result_df = pd.DataFrame(target_info,columns=columns)
        result_df.to_excel('./idus/result_0520.xlsx',encoding='utf-8-sig', index=False)
        time.sleep(3)
        
print(dict)

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))
target_info['title'] = title_list
target_info['nickname'] = nickname_list
target_info['datetime'] = datetime_list
target_info['like'] = like_list
target_info['content'] = content_list


454


  0%|          | 0/454 [00:00<?, ?it/s]

향기로운 플라워타블렛과 섬유향수만들기
반짝반짝 크리스탈캔들패브릭미스트 클래스
경복궁역향기 가득 플라워 왁스 타블렛 만들기
달콤 디저트 까눌레 캔들톰과제리 치즈캔들 만들기
건대입구 젤캔들 홀더와 티라이트4개 원데이클래스
건대입구 플라워 디퓨저  왁스타블렛 세트 원데이클래스
젤 캔들 홀더  티라이트 캔들 4구
왁스타블렛 2구 만들기 티라이트 캔들 증정
시간협의필수 보석은 역시 원석 향기로운 젬스톤 원석캔들 만들기
원목의 감성을 느낄 수 있는 소이왁스 방향제 만들기
원목과 향기 드라이플라워가 만난 하나뿐인 방향제
홍대 아름다운 향기오브제 왁스타블렛과 롤온향수 클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
귀여운 케이크캔들 디저트캔들 오브제캔들 원데이로 만들어요
귀여운 케이크캔들 디저트캔들 오브제캔들 원데이로 만들어요
귀여운 케이크캔들 디저트캔들 오브제캔들 원데이로 만들어요
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
나만의 탄생화 향수 만들기
감성오브제

In [ ]:
import pandas as pd
columns = ['title', 'nickname', 'datetime', 'like', 'content']
result_df = pd.DataFrame(target_info,columns=columns)
result_df.to_excel('./idus/result.xlsx',encoding='utf-8-sig', index=False)
result_df